In [12]:
# Dependencies
import requests
import numpy as np
import pandas as pd
from census import Census
import matplotlib.pyplot as plt

In [13]:
# Import U.S. Census API Key
from config import api_key

# Create an instance of the Census library
c = Census(
    api_key,
    year = 2021
)

In [23]:
slc_zipcodes = ["84101", "84102", "84103", "84104", "84105", "84106", "84107", "84108", "84109", "84110", "84111", "84112", "84113", "84114", "84115", "84116", "84117", "84118", "84119", "84120", "84121", "84122", "84123", "84124", "84125", "84126", "84127", "84128", "84130", "84131", "84132", "84133", "84134", "84136", "84138", "84139", "84141", "84143", "84145", "84147", "84148", "84150", "84151", "84152", "84157", "84158", "84165", "84170", "84171", "84180", "84184", "84189"]

# Run Census Search to retrieve data for Utah and Salt Lake City (2021 ACS5 Census)
census_data = c.acs5.get(
    (
        "NAME",
        "B19013_001E",
        "B01003_001E",
        "B01002_001E",
        "B19301_001E",
        "B17001_002E"
    ),
    {'for': 'zip code tabulation area:' + ','.join(slc_zipcodes)}
)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column renaming
census_pd = census_pd.rename(
    columns = {
        "B01003_001E": "Population",
        "B01002_001E": "Median Age",
        "B19013_001E": "Household Income",
        "B19301_001E": "Per Capita Income",
        "B17001_002E": "Poverty Count",
        "NAME": "Name",
        "zip code tabulation area": "Zipcode"
    }
)

# Add a Poverty Rate column (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Configure the final DataFrame
census_pd = census_pd[
    [
        "Name",
        "Population",
        "Median Age",
        "Household Income",
        "Per Capita Income",
        "Poverty Count",
        "Poverty Rate"
    ]
]

# Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_pd)}")
census_pd.head()

Number of rows in the DataFrame: 26


,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,ZCTA5 84101,6422.0,35.8,50000.0,52194.0,894.0,13.920897
1,ZCTA5 84102,18995.0,28.9,46906.0,38383.0,5188.0,27.312451
2,ZCTA5 84103,22888.0,36.8,73911.0,68737.0,2248.0,9.821741
3,ZCTA5 84104,24809.0,31.0,53401.0,21765.0,3495.0,14.087629
4,ZCTA5 84105,23117.0,35.2,86160.0,52499.0,2081.0,9.002033


In [22]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)